In [1]:
import os
from datetime import datetime
from dataloader import SeqDataLoader
import time
import numpy as np
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn
import torch.nn.functional as F

num_folds = 20
data_dir = 'data/data2013/eeg_fpz_cz'
classes = ['W', 'N1', 'N2', 'N3', 'REM']
checkpoint_dir = 'checkpoints-seq2seq-sleep-EDF'
output_dir = 'outputs_2013/outputs_eeg_fpz_cz'

kargs = {
    'epochs': 120,
    'batch_size': 20,
    'num_units': 128,
    'embed_size': 10,
    'input_depth': 3000,
    'n_channels': 100,
    'bidirectional': True,
    'use_attention': True,
    'lstm_layers': 2,
    'attention_size': 64,
    'beam_width': 4,
    'use_beamsearch_decode': False,
    'max_time_step': 10,
    'output_max_length': 10 + 2,
    'akara2017': True,
    'test_step': 5
}

path, channel_ename = os.path.split(data_dir)
traindata_dir = os.path.join(os.path.abspath(os.path.join(data_dir, os.pardir)),'traindata/')

print(path)
print(channel_ename)
print(traindata_dir)

print((str(datetime.now())))

fold_idx = 0

start_time_fold_i = time.time()
data_loader = SeqDataLoader(data_dir, num_folds, fold_idx, classes=classes)
X_train, y_train, X_test, y_test = data_loader.load_data(seq_len=kargs['max_time_step'])

print(start_time_fold_i)
print(X_train)
print(y_train)
print(X_test)
print(y_test)

# preprocessing
char2numY = dict(list(zip(classes, list(range(len(classes))))))        
pre_f1_macro = 0

print(char2numY)

# <SOD> is a token to show start of decoding  and <EOD> is a token to indicate end of decoding
char2numY['<SOD>'] = len(char2numY)
char2numY['<EOD>'] = len(char2numY)
num2charY = dict(list(zip(list(char2numY.values()), list(char2numY.keys()))))

print(num2charY)

nums = []
for cl in classes:
    nums.append(len(np.where(y_train == char2numY[cl])[0]))

print(nums)
if (os.path.exists(traindata_dir) == False):
    os.mkdir(traindata_dir)
fname = os.path.join(traindata_dir,'trainData_'+channel_ename+'_SMOTE_all_10s_f'+str(fold_idx)+'.npz')

if (os.path.isfile(fname)):
    X_train, y_train,_ = data_loader.load_npz_file(fname)
else:
    # oversampling
    n_osamples = nums[2] - 7000
    ratio = {0: n_osamples if nums[0] < n_osamples else nums[0], 1: n_osamples if nums[1] < n_osamples else nums[1],
                2: nums[2], 3: n_osamples if nums[3] < n_osamples else nums[3], 4: n_osamples if nums[4] < n_osamples else nums[4]}

    sm = SMOTE(random_state=12, sampling_strategy=ratio)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    data_loader.save_to_npz_file(X_train, y_train,data_loader.sampling_rate,fname)

    print(n_osamples)
    print(ratio)

    X_train = X_train[:(X_train.shape[0] // kargs['max_time_step']) * kargs['max_time_step'], :]
y_train = y_train[:(X_train.shape[0] // kargs['max_time_step']) * kargs['max_time_step']]

X_train = np.reshape(X_train,[-1,X_test.shape[1],X_test.shape[2]])
y_train = np.reshape(y_train,[-1,y_test.shape[1],])

print(X_train.shape)
print(y_train.shape)

# shuffle training data_2013
permute = np.random.permutation(len(y_train))
X_train = X_train[permute]
y_train = y_train[permute]

# # add '<SOD>' to the beginning of each label sequence, and '<EOD>' to the end of each label sequence (both for training and test sets)
# tar_train= [[char2numY['<SOD>']] + [y_ for y_ in date] + [char2numY['<EOD>']] for date in y_train]
# tar_train = np.array(tar_train)

# tar_test= [[char2numY['<SOD>']] + [y_ for y_ in date] + [char2numY['<EOD>']] for date in y_test]
# tar_test = np.array(tar_test)

c:\Users\user\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


data/data2013
eeg_fpz_cz
d:\sejin\GoodSleep\data\data2013\traindata/
2023-12-08 13:57:30.276860

========== [Fold-0] ==========

Load training set:
Loading data/data2013/eeg_fpz_cz\SC4631E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4151E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4042E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4722E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4812G0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4701E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4181E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4071E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4182E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4502E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4622E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4411E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4672G0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4711E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4292G0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4141E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4581G0.npz ...
Lo

ValueError: cannot reshape array of size 900831000 into shape (10,3000)

In [4]:
X_train=torch.tensor(X_train[:X_train.shape[0]//4])
y_train=torch.tensor(y_train[:y_train.shape[0]//4])
print(X_train.shape)
print(X_train[:10])
print(y_train.shape)
print(y_train[:10])

torch.Size([7547, 10, 3000])
torch.Size([7547, 10])


In [5]:
cnn1 = nn.Sequential(
    nn.Conv1d(3000, 64, 50, stride=6, padding=25),
    nn.ReLU(),
    nn.MaxPool1d(8, stride=8, padding=4),
    nn.Dropout(0.5),
    nn.Conv1d(64, 128, 8, stride=1, padding='same'),
    nn.ReLU(),
    nn.Conv1d(128, 128, 8, stride=1, padding='same'),
    nn.ReLU(),
    nn.Conv1d(128, 128, 8, stride=1, padding='same'),
    nn.ReLU(),
    nn.MaxPool1d(4, stride=4, padding=2),
)

cnn2 = nn.Sequential(
    nn.Conv1d(3000, 64, 400, stride=50, padding=200),
    nn.ReLU(),
    nn.MaxPool1d(4, stride=4, padding=2),
    nn.Dropout(0.5),
    nn.Conv1d(64, 128, 6, stride=1, padding='same'),
    nn.ReLU(),
    nn.Conv1d(128, 128, 6, stride=1, padding='same'),
    nn.ReLU(),
    nn.Conv1d(128, 128, 6, stride=1, padding='same'),
    nn.ReLU(),
    nn.MaxPool1d(2, stride=2, padding=1),
)

X_train = X_train.unsqueeze(-1)
new_x = X_train[:,0]
print(new_x.shape)

out1 = cnn1(new_x)
print(out1.shape)
out2 = cnn2(new_x)
print(out2.shape)

torch.Size([7547, 3000, 1])


c:\Users\user\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:294: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ..\aten\src\ATen\native\Convolution.cpp:660.)
  return F.conv1d(input, weight, bias, self.stride,


torch.Size([7547, 128, 1])
torch.Size([7547, 128, 1])


In [6]:
# Flatten
flatten_out1 = torch.flatten(out1, start_dim=1)
print(flatten_out1.shape)

# Flatten again
flatten_out2 = torch.flatten(out2, start_dim=1)
print(flatten_out2.shape)

torch.Size([7547, 128])
torch.Size([7547, 128])


In [7]:
# Concat
flatten_out = torch.cat((flatten_out1, flatten_out2), dim=-1)
flatten_out = F.dropout(flatten_out, 0.5)
print(flatten_out.shape)


torch.Size([7547, 256])


In [8]:
class CNN(nn.Module):
    def __init__(self, **kargs):
        super().__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv1d(3000, 64, 50, stride=6, padding=25),
            nn.ReLU(),
            nn.MaxPool1d(8, stride=8, padding=4),
            nn.Dropout(0.5),
            nn.Conv1d(64, 128, 8, stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv1d(128, 128, 8, stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv1d(128, 128, 8, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool1d(4, stride=4, padding=2),
        )

        self.cnn2 = nn.Sequential(
            nn.Conv1d(3000, 64, 400, stride=50, padding=200),
            nn.ReLU(),
            nn.MaxPool1d(4, stride=4, padding=2),
            nn.Dropout(0.5),
            nn.Conv1d(64, 128, 6, stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv1d(128, 128, 6, stride=1, padding='same'),
            nn.ReLU(),
            nn.Conv1d(128, 128, 6, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2, padding=1),
        )

    def forward(self, x):
        out1 = self.cnn1(x)
        out2 = self.cnn2(x)

        # Flatten
        flatten_out1 = torch.flatten(out1, start_dim=1)
        flatten_out2 = torch.flatten(out2, start_dim=1)

        # Concat
        flatten_out = torch.cat((flatten_out1, flatten_out2), dim=-1)
        flatten_out = F.dropout(flatten_out, 0.5)
        return flatten_out

In [9]:
X_train[:, 0].shape

torch.Size([7547, 3000, 1])

In [10]:
cnn = CNN(**kargs)

In [11]:
x = []
for i in range(X_train.shape[1]):
    _x = cnn(X_train[:, i])
    _x = _x.unsqueeze(1)
    x.append(_x)

In [12]:
x = torch.cat(x, dim = 1)
x.shape

torch.Size([7547, 10, 256])

In [13]:
lstm = nn.LSTM(2 * kargs['num_units'], kargs['num_units'], num_layers = kargs['lstm_layers'], bidirectional = True, batch_first = True)

encoder_outputs, (encoder_state, _) = lstm(x)
# 마지막 레이어의 양방향 hidden state 가져오기
hidden_state = encoder_state[-2:, :, :]

# 양방향 hidden state 연결
hidden_state = hidden_state.transpose(0, 1).contiguous().view(x.size(0), -1)
print(encoder_outputs.shape)
print(hidden_state.shape)

torch.Size([7547, 10, 256])
torch.Size([7547, 256])


In [14]:
x.shape

torch.Size([7547, 10, 256])

In [15]:
lstm_f = nn.LSTM(2 * kargs['num_units'], kargs['num_units'], batch_first = True)
lstm_b = nn.LSTM(2 * kargs['num_units'], kargs['num_units'], batch_first = True)


f, (fh, _) = lstm_f(x)
x_reversed = torch.flip(x, [1])
b, (bh, _) = lstm_b(x_reversed)
fh = fh.squeeze(0)
bh = bh.squeeze(0)
encoder_output = torch.cat([f, b], dim=-1)
h = torch.cat([fh, bh], dim=-1)
print(encoder_output.shape)
print(h.shape)

torch.Size([7547, 10, 256])
torch.Size([7547, 256])


In [16]:
NUM_CLASS = 7
class Attention(nn.Module):
    def __init__(self, **kargs):
        super().__init__()
        self.We = nn.Linear(256, 256)
        self.Wh = nn.Linear(256, 256)
        self.tanh=torch.tanh
        self.softmax = torch.nn.functional.softmax

    # encoder_hidden과 decoder_hidden과 shape이 같아야 함
    def forward(self, encoder_hidden, decoder_hidden):
        WE = self.We(encoder_hidden)
        decoder_hidden = decoder_hidden.unsqueeze(1)
        WH = self.Wh(decoder_hidden)
        x = WE + WH
        f = self.tanh(x)
        alpha = self.softmax(f, dim=-1)
        c = alpha * encoder_hidden
        c = torch.sum(c, dim=1)
        return c

class Decoder(nn.Module):
    def __init__(self, **kargs):
        super().__init__()
        
        self.attention = Attention(**kargs)
        self.lstm = nn.LSTM(263, kargs['num_units'], num_layers = kargs['lstm_layers'], bidirectional = True, batch_first = True)
        self.classes = nn.Linear(2 * kargs['num_units'], NUM_CLASS)
        # self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, decoder_input, prev_decoder_hidden, encoder_hidden):
        c = self.attention(encoder_hidden, prev_decoder_hidden)
        x = torch.cat([c, decoder_input], dim=-1)
        x = x.unsqueeze(1)
        x, h = self.lstm(x) # .squeeze(1)
        prediction = self.classes(x)
        return prediction, h

In [32]:
y_train.shape

torch.Size([7547, 10])

In [ ]:
classes = ['W', 'N1', 'N2', 'N3', 'REM']

In [38]:
y_train

tensor([[3, 3, 2,  ..., 3, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 0, 1,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [4, 4, 4,  ..., 4, 4, 4],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [39]:
# # convert to long tensor
# y_train = y_train.long()

# # one-hot encoding for each row
# one_hot_y_train = torch.stack([F.one_hot(row, num_classes=5) for row in y_train])

# print(one_hot_y_train)
# one_hot_y_train.shape # torch.Size([7547, 10, 5])

tensor([[[0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 1, 0, 0],
         ...,
         [0, 0, 0, 1, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         ...,
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0],
         ...,
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0]],

        ...,

        [[1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         ...,
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 1],
         ...,
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 1]],

        [[1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         ...,
         [1, 0, 0, 0, 0],
        

In [ ]:
# 실제 inference할 때는 feature = [1, 10, 3000, 1]으로 하면 될듯

# one hot encodding을 사용할 때는 output layer의 output_size=label
# 결과 shape (batch_size, seq_len, label)

In [44]:
decoder = Decoder(**kargs)
decoder_input = F.one_hot(torch.tensor(char2numY['<SOD>']), num_classes=7).float()
decoder_input = decoder_input.expand(X_train.shape[0], -1)  # <SOD>로 시작
pred, h = decoder(decoder_input, hidden_state, encoder_outputs)
pred = pred.squeeze(1)  # Remove the second dimension

In [47]:
decoder_input.shape

torch.Size([7547, 7])

In [45]:
pred.shape

torch.Size([7547, 7])

In [46]:
y_train.shape

torch.Size([7547, 10])

In [30]:
from torch.nn.functional import cross_entropy
from torch import optim


decoder_input = F.one_hot(torch.tensor(char2numY['<SOD>']), num_classes=7).float()
decoder_input = decoder_input.expand(X_train.shape[0], -1)  # <SOD>로 시작
# print(decoder_input.shape)
decoder = Decoder(**kargs)
optimizer = optim.Adam(decoder.parameters())  # Define an optimizer

pred, h = decoder(decoder_input, hidden_state, encoder_outputs)
_loss = 0
for t in range(10):
    pred, h = decoder(decoder_input, hidden_state, encoder_outputs)
    pred = pred.squeeze(1)  # Remove the second dimension
    _loss = cross_entropy(pred, y_train[:, t].long())
    print(_loss)
    _loss.backward()
    optimizer.step()

tensor(1.9337, grad_fn=<NllLossBackward>)
tensor(1.9284, grad_fn=<NllLossBackward>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved variables after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved variables after calling backward.

In [ ]:
total_time = time.time()

min_loss = 1e10
min_epoch = 0

for e in range(EPOCHS):
    start_time = time.time()
    
    for i,x in enumerate(train_seq_dataset):
        train_step(x)
        
    for i,x in enumerate(valid_seq_dataset):
        valid_step(x)
    
    ipd.clear_output(wait=True)
    print(f"{e+1}/{EPOCHS}, loss={loss.result():.8f}, train acc={acc.result()*100:.2f}%,")
    print(f"validation: loss={valid_loss.result():.8f}, acc={valid_acc.result()*100:.2f}%, {time.time()-start_time:.2f} sec/epoch, totally {time.time()-total_time:.2f} seconds")
    print(f"\tbest valid loss = {min_loss:.8f} at epoch-{min_epoch}")
    
    if min_loss > valid_loss.result():
        min_loss = valid_loss.result()
        min_epoch = e
        # BEST MODEL만 저장
#         encoder.save_weights(f"../weights/sleepeegnet/encoder-{SEED}")
#         decoder.save_weights(f"../weights/sleepeegnet/decoder-{SEED}")
    
    loss.reset_states()
    acc.reset_states()
    valid_loss.reset_states()
    valid_acc.reset_states()